In [4]:
import pyedflib
import numpy as np
import h5py

import csv
import math
#pyedflib.close_file("./sleepdata/SLP0001.edf")

import pandas as pd





'''
data_path = './sleepdata/SLP0001.edf'   # 存放数据的具体位置，需要改成自己数据存放的地方
signal_name  = 'C4A1'     # 所选的通道名称
raw_data = mne.io.read_raw_edf(dataset_path, preload=True)
# preload: 如果为True，则数据将被预加载到内存中(这样可以加快数据的索引), 默认为False
raw.pick_channels([signal_name])
eeg = raw.to_data_frame()   # 将读取的数据转换成pandas的DataFrame数据格式
eeg = list(eeg.values[:,1])  #转换成numpy的特有数据格式
'''
edffile = ["./sleepdata/SLP0001.edf",
               "./sleepdata/SLP0002.edf",
               "./sleepdata/SLP0003.edf",
               "./sleepdata/SLP0004.edf",
               "./sleepdata/SLP0005.edf"]
h5file = ["./sleepdata/h5data/slp0001.h5",
               "./sleepdata/h5data/slp0002.h5",
               "./sleepdata/h5data/slp0003.h5",
               "./sleepdata/h5data/slp0004.h5",
               "./sleepdata/h5data/slp0005.h5"]
complexd_h5file = ["./sleepdata/h5data/complexd_slp0001.h5",
               "./sleepdata/h5data/complexd_slp0002.h5",
               "./sleepdata/h5data/complexd_slp0003.h5",
               "./sleepdata/h5data/complexd_slp0004.h5",
               "./sleepdata/h5data/complexd_slp0005.h5"]
labelfile = ["./sleepdata/lable/slp0001-labels.csv",
                 "./sleepdata/lable/slp0001-labels.csv",
                 "./sleepdata/lable/slp0001-labels.csv",
                 "./sleepdata/lable/slp0001-labels.csv",
                 "./sleepdata/lable/slp0001-labels.csv"]
channels = ['ECG','EMG','EOG-R','EOG-L','O2-A1','C4-A1','F4-A1']


#降采样
def downsample(signals,signal_size,oldfreq,newfreq):

    index = pd.date_range('1/1/2000', periods=signal_size, freq=oldfreq)  #这个起始时间任意指定，freq为其频率
    data=pd.Series(signals,index=index)
    data_obj = data.resample(rule=newfreq,kind = 'period').mean().to_numpy()  #第一个为抽样频率，label表示左右开闭区间
    print(len(data_obj))
    return data_obj

#转换文件格式
def loadData(edffile,h5file,labelfile):
    #对所有文件进行转换
    for filei in range(5):
        fileedf = pyedflib.EdfReader(edffile[filei])
        fileh5 = h5py.File(h5file[filei],'w')
        signal_headers = fileedf.getSignalHeaders()
        #print("headers:",signal_headers)
        #print("参数",signal_headers[filei]['sample_rate'],signal_headers[filei]['digital_max'],signal_headers[filei]['digital_min'])
        fileh5['digital_max'] = np.array([signal_headers[filei]['digital_max']])
        fileh5['digital_min'] = np.array([signal_headers[filei]['digital_min']])
        #print("参数结果",fileh5['sample_rate'].value,fileh5['digital_max'].value,fileh5['digital_min'].value)

        row = 0
        for signi in list(range(len(signal_headers))):
            column = 120*30 #一个30秒epoch的列数
            size = fileedf.readSignal(signi,0,None,False).size  #获取数据点总数
            row = size//column ##行数
            oldsignals=fileedf.readSignal(signi,0,size,False)
            newsignals=downsample(oldsignals,size,'4L','8.333334L')
            newSignal=len(newsignals)//column * column ##epochs总数
            fileh5[str(signal_headers[signi]['label'])] = newsignals[0:newSignal].reshape(-1,column)  #存储数据点
        fileh5['sample_rate'] = np.array([120],dtype=int)

        #将标签加入数据文件中
        with open(labelfile[filei],"r") as f:
            csv_read = csv.reader(f)
            labels = []
            #print("行数",row)
            for labelsum,label in enumerate(csv_read):
                if labelsum != row :
                    labels.append(label[0])  #或许对应数量的标签
                else:
                    break
            labels = np.array(labels,dtype='int32').reshape(-1,1) #转换类型
            print(labels.shape)
            fileh5['labels'] = labels
        #print(fileh5['labels'])
        #print("---------------")
        fileedf.close()
print("开始")
loadData(edffile,h5file,labelfile)
print("结束")



def getData(complexd_h5file,channels,sample_rate='sample_rate',digital_max='digital_max',digital_min='digital_min',labels='labels'):

    for filei in range(5):

        fileh5 = h5py.File(complexd_h5file[filei], 'r')

        print(type(fileh5.keys()), fileh5.keys())
        print("------------------------")
        fileh5.close()
    return

# print("开始")
# getData(complexd_h5file)
# print("结束")

def pcaWhite(X):
    X -= np.mean(X, axis = 0) # 减去均值，使得以0为中心
    cov = np.dot(X.T, X) / X.shape[0] #计算协方差矩阵
    U,S,V = np.linalg.svd(cov) #矩阵的奇异值分解
    Xrot = np.dot(X, U)
    Xwhite = Xrot / np.sqrt(S + 1e-5) #加上1e-5是为了防止出现分母为0的异常
    return X

##将实数转化为复数
def complexedValue(complexd_h5file, channels,white = 0,sample_rate='sample_rate',digital_max='digital_max',digital_min='digital_min',labels='labels'):


    for filei in range(5):


        complexd_fileh5 = h5py.File(complexd_h5file[filei],'w')
        fileh5 = h5py.File(h5file[filei],'r')

        #将信息放入新文件中
        complexd_fileh5[sample_rate] = np.array([fileh5[sample_rate][0]])
        complexd_fileh5[digital_max] = np.array([fileh5[digital_max][0]])
        complexd_fileh5[digital_min] = np.array([fileh5[digital_min][0]])
        complexd_fileh5[labels] = fileh5[labels].value
        #print(complexd_fileh5['labels'].value[1:10,:],fileh5['labels'].value[1:10,:])
        #z(i) = cosx + i*sinx  分别放在数组中
        digital_max_num = np.array(fileh5[digital_max][0])
        digital_min_num = np.array(fileh5[digital_min][0])

        #遍历所有通道，并将数据转换为复值形式
        for channel in channels:
            channel_data = fileh5[channel].value
            angle =  math.pi*(channel_data - digital_min_num)/(digital_max_num-digital_min_num)
            complexd_fileh5[channel+"_angle"] = angle
            anglecos=np.cos(angle)  #reshape
            anglesin=np.sin(angle)
            #print("白化开始")
            if white == 1:
                #白化处理
                complexd_fileh5[channel+"_cos"] = pcaWhite(anglecos)
                complexd_fileh5[channel+"_sin"] = pcaWhite(anglesin)

            else:
                complexd_fileh5[channel+"_cos"] = anglecos
                complexd_fileh5[channel+"_sin"] = anglesin
                print(len(complexd_fileh5[channel+"_cos"][0]))


            # print('白化-------------')
            # complexd_fileh5[channel+"_cos"] = pcaWhite(np.cos(angle))    #白化
            # complexd_fileh5[channel+"_sin"] = pcaWhite(np.sin(angle))    #白化
            #print('白化结束---------')
        fileh5.close()
        complexd_fileh5.close()


# print("开始")
# complexedValue(complexd_h5file, channels,white=0)
# print("结束")



开始
2860800
2860800
2860800
2860800
2860800
2860800
2860800
(1655, 1)
2822400
2822400
2822400
2822400
2822400
2822400
2822400
(1633, 1)
2749440
2749440
2749440
2749440
2749440
2749440
2749440
(1591, 1)
3033600
3033600
3033600
3033600
3033600
3033600
3033600
(1755, 1)
2968320
2968320
2968320
2968320
2968320
2968320
2968320
(1717, 1)
结束
